In [ ]:
!pip install tensorflow
!pip install imblearn xgboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load and preprocess the data
def load_and_preprocess_data(file_path):
    # Load the data
    df = pd.read_csv(file_path)

    # Separate features and target
    X = df.drop('Class', axis=1)
    y = df['Class']

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply SMOTE to handle class imbalance
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    return X_train_resampled, X_test_scaled, y_train_resampled, y_test

# XGBoost model
def train_xgboost(X_train, y_train):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    model.fit(X_train, y_train)
    return model

# Neural Network model
def train_neural_network(X_train, y_train, input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    return model

# Ensemble predictions
def ensemble_predictions(xgb_model, nn_model, X_test):
    xgb_pred = xgb_model.predict_proba(X_test)[:, 1]
    nn_pred = nn_model.predict(X_test).flatten()
    ensemble_pred = (xgb_pred + nn_pred) / 2
    return ensemble_pred

# Main function
def main():
    # Load and preprocess data
    X_train, X_test, y_train, y_test = load_and_preprocess_data('/content/creditcard.csv')

    # Train XGBoost model
    xgb_model = train_xgboost(X_train, y_train)

    # Train Neural Network model
    nn_model = train_neural_network(X_train, y_train, X_train.shape[1])

    # Make ensemble predictions
    y_pred_proba = ensemble_predictions(xgb_model, nn_model, X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Evaluate the model
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nROC AUC Score:")
    print(roc_auc_score(y_test, y_pred_proba))

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:46:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11373/11373 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9665 - loss: 0.0867 - val_accuracy: 0.9967 - val_loss: 0.0104
Epoch 2/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - accuracy: 0.9964 - loss: 0.0125 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 3/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9973 - loss: 0.0099 - val_accuracy: 0.9995 - val_loss: 0.0053
Epoch 4/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9981 - loss: 0.0073 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 5/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - accuracy: 0.9983 - loss: 0.0067 - val_accuracy: 0.9994 - val_loss: 0.0022
Epoch 6/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9984 - loss: 0.0061 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 7/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - accuracy: 0.9986 - loss: 0.0060 - val_accuracy: 0.9999 - val_loss: 0.0011
Epoch 8/10
11373/11373 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - accuracy: 0.9987 - lo